# Customized Random Forest Classifier Model 

## Base algorithm

Premises: all features must be numeric of type integer.

1. Build a decision tree.

    1.1. Randomized bootstrap.
    
    1.1.1. Sample from training dataset: $X_T$ wiht size $N_T$.
    
    1.1.2. Sample from training dataset: $X_V$ wiht size $N_V \geq N_T$ .
        
    1.1.3. Set randomly the number of features to use: $n_F$.
    
    1.1.4. Set randomly witch features to use: $F$.
    
    1.1.5. Set a metric type (Ac., F1, FPR etc.): $M$.
    
    1.1.6. Set the limit for drop trees: $D$.
    
    1.1.7. Set how much trees: $T$
    
    1.1.8. Set decrease step to threshold for drop: $\Delta Th$
    
2. Forest creation algorithm:
    
```text

function splitData(feat,dataset):
    *Split the data by median.
    Ensuring: half/half split.
        Blanced calsses*
    return ds_a, ds_b, median
    
function validationTree(tree,ds,metric):
    *Test the tree with given datset ds.
    Return the score based on given metric*
    return score

function sampleFeats(n_F_min,n_F_max):
    n_F = random(n_F_min,n_F_max)
    return sample(Feature_List,n_F)
    
function genLeaf(target_col):
    return {
        'leaf' : {
            'Class_0': target_col.rate('Class_0'),
            'Class_1': target_col.rate('Class_1')
        }
    }

function growTree(F,ds):

    if size(ds.target.unique) == 1:
        return genLeaf(ds.target)
        
    feat = F.pop(0)
    ds_a, ds_b, split_val = splitData(feat,ds)

    return {
            feat: {
                'split': {
                    'split_val': split_val,
                    'A' : genLeaf(ds_a.target),
                    'B' : genLeaf(ds_b.target)
                } if F is null else {
                    'split_val': split_val,
                    'A' : drowTree(F,ds_a),
                    'B' : growTree(F,ds_b)
                }
            }
        }

Forest = []
for t in T:
    X_T, X_V = split_train_test(dataset, train_size=N_T, test_size=N_V)
    droped_trees = 0
    Threshold_for_drop = 1 - delta_Th
    Pass = False
    while not Pass:
        F = sampleFeats(n_F_min,n_F_max)
        Tree = growTree(F,X_T)
        if validationTree(Tree,X_V,M) < Threshold_for_drop:
            droped_trees += 1
        else:
            Pass = True
        if droped_trees >= D:
            Threshold_for_drop -= delta_Th
            droped_trees = 0
    Forest.append(Tree)

```

3. Notes.

    3.1. Need some adjustments for inbalanced dataset.

- Author: Israel Oliveira [\[e-mail\]](mailto:'Israel%20Oliveira%20'<prof.israel@gmail.com>)

In [2]:
%load_ext watermark

In [3]:
import pandas as pd

In [4]:
# from tqdm import tqdm

# from glob import glob

# import matplotlib.pyplot as plt
# %matplotlib inline
# from matplotlib import rcParams
# from cycler import cycler

# rcParams['figure.figsize'] = 12, 8 # 18, 5
# rcParams['axes.spines.top'] = False
# rcParams['axes.spines.right'] = False
# rcParams['axes.grid'] = True
# rcParams['axes.prop_cycle'] = cycler(color=['#365977'])
# rcParams['lines.linewidth'] = 2.5

# import seaborn as sns
# sns.set_theme()

# pd.set_option("max_columns", None)
# pd.set_option("max_rows", None)
# pd.set_option('display.max_colwidth', None)

# from IPython.display import Markdown, display
# def md(arg):
#     display(Markdown(arg))

# from pandas_profiling import ProfileReport
# #report = ProfileReport(#DataFrame here#, minimal=True)
# #report.to

# import pyarrow.parquet as pq
# #df = pq.ParquetDataset(path_to_folder_with_parquets, filesystem=None).read_pandas().to_pandas()

# import json
# def open_file_json(path,mode='r',var=None):
#     if mode == 'w':
#         with open(path,'w') as f:
#             json.dump(var, f)
#     if mode == 'r':
#         with open(path,'r') as f:
#             return json.load(f)

# import functools
# import operator
# def flat(a):
#     return functools.reduce(operator.iconcat, a, [])

# import json
# from glob import glob
# from typing import NewType


# DictsPathType = NewType("DictsPath", str)


# def open_file_json(path):
#     with open(path, "r") as f:
#         return json.load(f)

# class LoadDicts:
#     def __init__(self, dict_path: DictsPathType = "./data"):
#         Dicts_glob = glob(f"{dict_path}/*.json")
#         self.List = []
#         self.Dict = {}
#         for path_json in Dicts_glob:
#             name = path_json.split("/")[-1].replace(".json", "")
#             self.List.append(name)
#             self.Dict[name] = open_file_json(path_json)
#             setattr(self, name, self.Dict[name])


In [5]:
# Run this cell before close.
%watermark -d --iversion -b -r -g -m -v
!cat /proc/cpuinfo |grep 'model name'|head -n 1 |sed -e 's/model\ name/CPU/'
!free -h |cut -d'i' -f1  |grep -v total

Python implementation: CPython
Python version       : 3.9.6
IPython version      : 7.25.0

Compiler    : GCC 8.3.0
OS          : Linux
Release     : 5.11.0-7620-generic
Machine     : x86_64
Processor   : 
CPU cores   : 4
Architecture: 64bit

Git hash: ed3aba7692c8a9756f12bca72c13a990d76a3939

Git repo: https://github.com/ysraell/creditcardfraud.git

Git branch: main

pandas: 1.3.1

CPU	: Intel(R) Core(TM) i7-7500U CPU @ 2.70GHz
Mem:           15G
Swap:         4.0G


In [6]:
ds = pd.read_csv('/work/data/creditcard_trans_int.csv')
target_col = 'Class'
ds_cols = ['V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10', 'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20', 'V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28', 'Amount']

In [11]:
ds[target_col].value_counts()[1]

492

In [14]:
#
N_fraud_test = 200
N_truth_test = int(2e4)
N_truth_train = int(2e5)

# How much trees:
T = 100

# Parameters for bootstrap

# Data for each tree:
ds_size = 11*(ds[target_col].value_counts()[1] - N_fraud_test)

# How much features:
n_F_max = len(ds_cols)
n_F_min = n_F_max//2

# Droped trees limit:
D = 100

# Delta Threshold
delta_Th = 0.02

# Metric

3212

In [15]:
n_F_min

14